# Intro to Pandas, Reading Data, and Plotting

The cell below loads up a few libraries and does some initialization.  In this notebook we'll do a few basic data manipulations and see the Pandas formatting for the first time and make some simple graphs.


In [ ]:
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')
# Seaborn is a plotting package for Pandas that we'll try out...
import seaborn as sns

# Make the fonts a little bigger..
font = {'size'   : 24}
matplotlib.rc('font', **font)
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['pdf.fonttype'] = 42

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Working with some real data and Pandas!

Opening and reading CSV files is very easy with Pandas [Read CSV Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [ ]:
# Open the NBA Salaries file.

df_nba = pd.read_csv("./data/nba_salaries.csv")

In [ ]:
# Display gives us a basic table.  Note that we can index and slice this in many different ways.
display(df_nba[:5])

In [ ]:
# Look at a specific person... a little clunky.
df_nba.loc[df_nba['PLAYER'] == "Stephen Curry"]

In [ ]:
# Can filter for a whole set
df_nba.loc[df_nba['POSITION'] == "PG"][:5]

# Again note that we can slice this different ways..

In [ ]:
# Can also see a team...
df_nba.loc[df_nba['TEAM'] == "New Orleans Pelicans"]

In [ ]:
# Look at all PGs sorted by salary...
df_nba.loc[df_nba['POSITION'] == "PG"].sort_values("'15-'16 SALARY", ascending=False)[:5]

Note that the above sorting does not happen *in place* unless we explicitly tell Pandas to do so -- [Documentation for sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)

In [ ]:
# Once we sort by values it does not stay the same unless we overwrite the table or do it in place...
df_nba.loc[df_nba['POSITION'] == "PG"][:5]

In [ ]:
# Maybe see what the position distribution is...
sns.countplot(df_nba['POSITION'])

## We can also use Pandas to read a CSV that is online... 


In [ ]:
#df_class_survey = pd.read_csv("./Data Science Day 1 Questions (Responses) - Form Responses 1.csv")

# We can also read directly from a google sheet if we want.  Note that at the end we have to add `/export?gid=1081980213&format=csv`
# The gid field tell us what sheet to load and the format gives us csv
df_class_survey = pd.read_csv("https://docs.google.com/spreadsheets/d/1d4C9HEIOkL7x_W4rYCRsflt_Mw7I6DGLUbAAUwwIqUM/export?gid=1081980213&format=csv")

In [ ]:
df_class_survey[:5]

In [ ]:
# Maybe see what the position distribution is...
sns.countplot(df_class_survey['I use Jupyter Notebooks'])


In [ ]:
# Maybe see what the position distribution is...
g = sns.countplot(df_class_survey['I use Jupyter Notebooks'])
g.set_xticklabels(g.get_xticklabels(),rotation=-85)
display(g)

# That was fun, let's try to read some books...

In [ ]:
from urllib.request import urlopen 
import re
def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

In [ ]:

# Read two books, fast!

huck_finn_url = 'https://www.inferentialthinking.com/data/huck_finn.txt'
huck_finn_text = read_url(huck_finn_url)
huck_finn_chapters = huck_finn_text.split('CHAPTER ')[44:]

little_women_url = 'https://www.inferentialthinking.com/data/little_women.txt'
little_women_text = read_url(little_women_url)
little_women_chapters = little_women_text.split('CHAPTER ')[1:]

In [ ]:
huck_finn_chapters[2]

In [ ]:
# Turn it into a data frame..
df_huck = pd.DataFrame(huck_finn_chapters, columns=["Text"])

In [ ]:
display(df_huck[:5])

In [ ]:
# Count how many times we see each character...
# Here we make a data frame out of a dictionary where the index is the column name
# and the values are the column
counts = pd.DataFrame({
        'Jim':np.char.count(huck_finn_chapters, 'Jim'),
        'Tom':np.char.count(huck_finn_chapters, 'Tom'),
        'Huck':np.char.count(huck_finn_chapters, 'Huck')
    })

In [ ]:
counts[:5]

In [ ]:
ax = counts.cumsum().plot(figsize=(10,8))
ax.set_xlabel("Chapter")
ax.set_ylabel("Number of Times")
#ax.set_ylim((-5,310))
plt.show()

There are lots of options for the figures ... Note that here we are using [Pandas Plot](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.plot.html) which is a wrapper around [MatPlot's Plot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.plot.html).

In [ ]:
ax = counts.cumsum().plot(figsize=(10,8), fontsize=(5),
                                          lw=2, 
                                          markersize=12,
                                          style=['X-','o-.','v--','s:','d:','*-.'])
ax.set_xlabel("Chapter")
ax.set_ylabel("Number of Times")
ax.set_ylim((-5,310))
plt.show()

In [ ]:
# Now for Little women...

people = ['Amy', 'Beth', 'Jo', 'Laurie', 'Meg']
people_counts = {pp: np.char.count(little_women_chapters, pp) for pp in people}


In [ ]:
people_counts.keys()

In [ ]:
people_counts['Beth']

In [ ]:
# Make a pandas table...
counts = pd.DataFrame(people_counts)
counts[:5]

In [ ]:
ax = counts.cumsum().plot(figsize=(10,8), fontsize=(15),
                                          lw=2, 
                                          markersize=12,
                                          style=['X-','o-.','v--','s:','d:','*-.'])
ax.set_xlabel("Chapter")
ax.set_ylabel("Number of Times")
plt.show()

## Something more fun...

Inspired by the [Inferential Thinking Book](https://www.inferentialthinking.com/chapters/01/3/2/Another_Kind_Of_Character) let's do some more analysis on the text that we have loaded up.

First let's count the number of periods and the total number of characters in each of the books.

In [ ]:
# Recall that each element in the array corresponds to a chapter.
print(huck_finn_chapters[0][:50])
print(little_women_chapters[0][:50])

In [ ]:
chars_periods_huck_finn = pd.DataFrame({
        'Huck Finn Chapter Length':[len(s) for s in huck_finn_chapters],
        'Number of Periods':np.char.count(huck_finn_chapters, '.')
        })
chars_periods_little_women = pd.DataFrame({
        'Little Women Chapter Length': [len(s) for s in little_women_chapters],
        'Number of Periods': np.char.count(little_women_chapters, '.')
        })

In [ ]:
display(chars_periods_huck_finn[:5])
display(chars_periods_little_women[:5])

What do we notice about the above?  It seems like *Little Women* is significantly longer per chapter than *Huck Finn*.  Let's try plotting this relationship on the same graph.

To do this we are going to use the `scatter` function from [MatPlotLib](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.scatter.html)

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(chars_periods_huck_finn["Number of Periods"], 
              chars_periods_huck_finn["Huck Finn Chapter Length"], 
              color='darkblue')

plt.scatter(chars_periods_little_women["Number of Periods"], 
              chars_periods_little_women["Little Women Chapter Length"], 
              color='gold')

plt.xlabel('Number of periods in chapter')
plt.ylabel('Number of characters in chapter')

The above plot shows us a few things:
1. Little Women is much longer on average than Huck
2. There seems to be a linear relationship between the number of characters and the number of periods

If we look at all the chapters that have 100 periods we see they have 10,000 - 15,000 characters.. or roughly 100-150 characters per sentence.  Seems like a Tweet.

In [ ]:
# Let's formally find the relationship...
from scipy import stats

# First let's make the tables the same..
chars_periods_huck_finn.columns = ['characters', 'periods']
chars_periods_little_women.columns = ['characters', 'periods']
display(chars_periods_huck_finn[:5])
len(chars_periods_huck_finn)

In [ ]:
# Now we are going to concatinate the data together -- this is our first join operation!

merged = pd.concat([chars_periods_huck_finn, chars_periods_little_women])
merged

In [ ]:
len(merged)

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(merged['periods'],merged['characters'])

In [ ]:
slope

In [ ]:
intercept

In [ ]:
r_value

In [ ]:
p_value

In [ ]:
std_err

In [ ]:
line = slope * merged['periods'] + intercept

Now we can add the line above to our plot using the [plot function](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.plot.html).

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(chars_periods_huck_finn["periods"], 
              chars_periods_huck_finn["characters"], 
              color='darkblue')

plt.scatter(chars_periods_little_women["periods"], 
              chars_periods_little_women["characters"], 
              color='gold')

plt.plot(merged['periods'], line, lw=1, ls=':')

plt.xlabel('Number of periods in chapter')
plt.ylabel('Number of characters in chapter')